Note that it might make more sense to just request cov(x,y), std(x), std(y) and compute correlation on requester's end instead of requesting the whole HUGE circuit of correlation. 

In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

This example is not necessary. In fact, a person can just request cov(x,y), std(x), and std(y). Then just compute correlation on his own as well, but here we show that the code is composable enough to do all at once. 

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [5]
selected_columns = ['x', 'y']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.correlation(x, y)

error = 0.01
# Prover/ data owner side
_, prover_model = computation_to_model(computation, precal_witness_path, True, selected_columns, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)


/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inf

==== Generate & Calibrate Setting ====
scale:  [5]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":1,"lookup_range":[-32360,32536],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":7872,"total_assignments":3691,"total_const_size":1319,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5,5],"module_sizes":{"kzg":[],"poseidon":[7872,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"Abs"],"required_range_checks":[[-16,16]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715154399878}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, selected_columns, error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 7.0670530796051025 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 0.71875
==== Generating Proof ====
proof:  {'instances': [['7469705c7b45875730aac31b009e8f9e9f9e57031bba148110877f78e4518c22', 'c1f946e3fd2b2229bfcc2dc9ff7f4dfc1fddad53635becb58d130bcae5504712', '0100000000000000000000000000000000000000000000000000000000000000', '1700000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x2f3267cc85cf8d450a1173d2965adf49b64e2bcb73a29ae02fa10c1a6150b3a424cd82cb0556c3fc8eb92d8512fa1bb30be186ddabb85baf85f3c992aa0cc7f01ee5e8d1b58e1e58dc2e4d1664a89da0331b8b8d37861d6bb876fdea9cc3ce7f097e13d4d80360d115cca485f87bb77df69d6c2593379b7164a983ef710031a60d3b9fb96d5d9b6e42680017972c7156c4d2ff8d9bc14fdb359c6bb8be0f40640636c75b1fde9eee43506975258d554a92374332f601d652462edaad08bbf79c19339a2d624468876686a285f65849ec8de8b2fb894395d4205ec755655039bb29f304072fd638c1aed2cf35541dbd73fd7bc88e32a5d0aeb88d1ee08e3339ef218f5170

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [0.71875]
